In [6]:
import os
import asyncio
import nest_asyncio
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool as langchain_tool
from langchain.agents import create_agent

llm = ChatOpenAI(
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("BASE_URL"),
    model="deepseek-chat",
    temperature=0.1,
)

# 定义工具
@langchain_tool
def search_information(query: str) -> str:
    """
    根据主题提供事实信息。用于回答如“法国首都”或“伦敦天气?”等问题。
    """
    print(f"\n‐‐‐ :tools:✅ 工具调用:search_information, 查询:'{query}' ‐‐‐")
    # 用预设结果模拟搜索工具
    simulated_results = {
        "weather in london": "伦敦当前天气多云,气温 15°C。",
        "capital of france": "法国的首都是巴黎。",
        "population of earth": "地球人口约 80 亿。",
        "tallest mountain": "珠穆朗玛峰是海最高的山峰。",
        "default": f"模拟搜索 '{query}':未找到具体信息,但该主题很有趣。"
    }
    result = simulated_results.get(query.lower(), simulated_results["default"])
    print(f"‐‐‐ 工具结果:{result} ‐‐‐")
    return result

tools = [search_information]

system_prompt = "你是一个乐于助人的助手。"

agent_executor = create_agent(
    model=llm,
    tools=tools,
    system_prompt=system_prompt,
)

async def run_agent_with_tool(query: str):
    """用 Agent 执行查询并打印最终回复。"""
    print(f"\n‐‐‐ :emoji: Agent 运行查询:'{query}' ‐‐‐")
    try:
        response = await agent_executor.ainvoke({
            "messages": [{"role": "user", "content": query}]
        })
        print("\n‐‐‐ ✅ Agent 最终回复 ‐‐‐")
        print(response)
    except Exception as e:
        print(f"\n:emoji: Agent 执行出错:{e}")

async def main():
    """并发运行多个Agent查询"""
    tasks = [
        run_agent_with_tool("法国的首都是什么?"),
        run_agent_with_tool("伦敦天气如何?"),
        run_agent_with_tool("说说狗的相关信息。") 
    ]
    await asyncio.gather(*tasks)

nest_asyncio.apply()
asyncio.run(main())




‐‐‐ :emoji: Agent 运行查询:'法国的首都是什么?' ‐‐‐

‐‐‐ :emoji: Agent 运行查询:'伦敦天气如何?' ‐‐‐

‐‐‐ :emoji: Agent 运行查询:'说说狗的相关信息。' ‐‐‐

‐‐‐ :tools:✅ 工具调用:search_information, 查询:'法国首都' ‐‐‐
‐‐‐ 工具结果:模拟搜索 '法国首都':未找到具体信息,但该主题很有趣。 ‐‐‐

‐‐‐ :tools:✅ 工具调用:search_information, 查询:'伦敦天气' ‐‐‐
‐‐‐ 工具结果:模拟搜索 '伦敦天气':未找到具体信息,但该主题很有趣。 ‐‐‐

‐‐‐ :tools:✅ 工具调用:search_information, 查询:'法国首都是哪个城市' ‐‐‐
‐‐‐ 工具结果:模拟搜索 '法国首都是哪个城市':未找到具体信息,但该主题很有趣。 ‐‐‐

‐‐‐ :tools:✅ 工具调用:search_information, 查询:'London weather today temperature forecast' ‐‐‐
‐‐‐ 工具结果:模拟搜索 'London weather today temperature forecast':未找到具体信息,但该主题很有趣。 ‐‐‐

‐‐‐ ✅ Agent 最终回复 ‐‐‐
{'messages': [HumanMessage(content='法国的首都是什么?', additional_kwargs={}, response_metadata={}, id='1c1dcef6-9644-4b46-b100-d96aba90bac8'), AIMessage(content='我来帮您查询法国的首都信息。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 324, 'total_tokens': 376, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, '